In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Apply Yellow-White Mask
def yw_mask(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    lower_yellow = np.array([20, 100, 100], dtype='uint8')
    upper_yellow = np.array([30, 255, 255], dtype='uint8')

    mask_yellow = cv2.inRange(img_hsv, lower_yellow, upper_yellow)
    mask_white = cv2.inRange(img_gray, 125, 255)
    mask_yw = cv2.bitwise_or(mask_white, mask_yellow)
    mask_yw_img = cv2.bitwise_and(img_gray, mask_yw)
    
    return mask_yw_img


# Add Gaussian Blur
def gauss_blur(mask_yw_img):
    return cv2.GaussianBlur(mask_yw_img, (5, 5), 0)


# Canny Edge Detection
def get_canny_edges(mask_yw_gauss):
    canny_edges = cv2.Canny(mask_yw_gauss, 5, 15)
    dilate = cv2.dilate(canny_edges, (5, 5), iterations=2)
    ero = cv2.erode(dilate, (5, 5), iterations=1)
    return ero


# Create and Apply ROI Mask
def apply_roi_mask(canny_edges):
    width, height = canny_edges.shape[1], canny_edges.shape[0]

    roi_mask = np.zeros_like(canny_edges)
    roi_corners = [np.array([
        [width/9, height],
        [width/2 - width/8, height/2 + height/5],
        [width/2 + width/8, height/2 + height/5],
        [width - width/9, height]
    ], dtype=np.int32)]

    cv2.fillPoly(roi_mask, roi_corners, 255)
    
    roi_canny = cv2.bitwise_and(canny_edges, roi_mask)
    return roi_mask, roi_canny


def make_line_coords(img, slope, intercept):
        y1 = img.shape[0]
        y2 = int(y1 * 0.7)
        x1 = int((y1 - intercept) / slope)
        x2 = int((y2 - intercept) / slope)
        
        return np.array([x1, y1, x2, y2])

# Average into 2 Lines
def avg_lines(img, lines): 
    left = [] 
    right = [] 
    
    for line in lines: 
        x1, y1, x2, y2 = line.reshape(4)
        slope, intercept = np.polyfit((x1, x2), (y1, y2), 1)
        
        if slope < 0:
            left.append((slope, intercept))    # Negative slope = left lane marking
        else:
            right.append((slope, intercept))   # Positive slope = right lane marking
    
    left_line = np.array([0, 0, 0, 0])
    right_line = np.array([0, 0, 0, 0])
    
    if left:
        left_avg = np.average(left, axis=0)
        left_line = make_line_coords(img, left_avg[0], left_avg[1])
    
    if right:
        right_avg = np.average(right, axis=0)
        right_line = make_line_coords(img, right_avg[0], right_avg[1])
    
    return np.array([left_line, right_line])


def group_lines(lines, distance_threshold):
    pass


# Hough Lines
def draw_hough_lines(img, roi_canny):
    dotted_img = np.zeros_like(img)
    line_img = np.zeros_like(img)
    lines = cv2.HoughLinesP(roi_canny,
                            2,
                            np.pi/180,
                            100,
                            minLineLength=40,
                            maxLineGap=100)
    
    #averaged_lines = avg_lines(img, lines)
    
    print(lines)
    for line in lines:
        x1, y1, x2, y2 = line[0]
        cv2.circle(dotted_img, (x1, y1), 5, (0, 255, 0), 2)
        cv2.line(line_img, (x1, y1), (x2, y2), (0, 0, 255), 5)

    final_img = cv2.addWeighted(img, 0.8, line_img, 1.0, 0.0)
    dotted_img = cv2.addWeighted(img, 0.8, dotted_img, 1.0, 0.0)
    return dotted_img, final_img

In [ ]:
"""img = cv2.imread("test_image.png")

mask_yw_img = yw_mask(img)
mask_yw_gauss = gauss_blur(mask_yw_img)
canny_edges = get_canny_edges(mask_yw_gauss)
roi_canny = roi_mask(canny_edges)
final_img = draw_hough_lines(img, roi_canny)

cv2.imshow("Hough Lines", final_img)

cv2.waitKey(10000)
cv2.destroyAllWindows()
cv2.waitKey(1)"""

In [ ]:
cap = cv2.VideoCapture("Colorado-Dashcam_edited.mp4")

while True:
    success, frame = cap.read()
    if not success:
        break
    
    intensity_matrix = np.ones(frame.shape, dtype='uint8') * 15
    bright_frame = cv2.add(frame, intensity_matrix)
    
    mask_yw_img = yw_mask(bright_frame)
    mask_yw_gauss = gauss_blur(mask_yw_img)
    canny_edges = get_canny_edges(mask_yw_gauss)
    roi_mask, roi_canny = apply_roi_mask(canny_edges)
    dotted_frame, final_frame = draw_hough_lines(frame, roi_canny)

    plt.figure()
    plt.imshow(mask_yw_img, cmap='gray')
    plt.show()

    plt.figure()
    plt.imshow(canny_edges, cmap='gray')
    plt.show()

    plt.figure()
    plt.imshow(roi_mask, cmap='gray')
    plt.show()
    
    plt.figure()
    plt.imshow(roi_canny, cmap='gray')
    plt.show()
    
    plt.figure()
    plt.imshow(cv2.cvtColor(final_frame, cv2.COLOR_BGR2RGB))
    plt.show()
    
    plt.figure()
    plt.imshow(cv2.cvtColor(dotted_frame, cv2.COLOR_BGR2RGB))
    plt.show()
    
    break
    
    cv2.imshow("Video", final_frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
cv2.waitKey(1)